In [94]:
import os

# for data aggregation.
import numpy as np
import pandas as pd
import geopandas as gpd
from geopy.distance import distance, lonlat

# for data visualisation.
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')
import deckgljupyter.Layer as deckgl
import seaborn as sns
import matplotlib.pyplot as plt
import deckgljupyter.Layer as deckgl
from deckgljupyter import Layer

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [96]:
access_token = 'pk.eyJ1IjoidG9wc29sMTciLCJhIjoiY2szOGQ3MWhoMDdtdDNqb3ZlNHBla2w4bSJ9.lgVDxlf8ndzEDe496xKi0Q'
view_options = {
    'center': [126.908, 37.132],
    'zoom': 12,
    'bearing': 0,
    'pitch': 60,
    'style': 'mapbox://styles/mapbox/dark-v9',
    'access_token': access_token
}

def get_random_color(opacity=1):
    """
        deckgl.Layer 파라미터 중 getColor 에 값을 주는데 쓰입니다.
    """
    rgba = np.random.choice(range(256), size=3).tolist()
    rgba.append(opacity*255)
    return rgba


def draw_path(path_list, name_list=None, colors=[], opacity=1): 
    """
        path_list 로 넘어온 path 들을 한 지도 안에 그립니다.
    """
    # 시각화할 데이터 형태로 가공
    data = []
    for i, path in enumerate(path_list):
        d = {
            'path': path,
            'color': get_random_color() if len(colors) == 0 else colors[i]
        }
        if name_list != None:
            d['name'] = name_list[i]
        data.append(d)
            
    # 지도 설정
    v = view_options.copy()
    v['center'] = np.mean(path_list[0], axis=0)
    
    # 지도그리기
    m = deckgl.Map(**view_options)
    m.add(deckgl.PathLayer(data,
                           getWidth=20,
                           getColor='obj => obj.color',
                           opacity=opacity,
                           pickable=True,
                           tooltip=['name']))
    
    m.show()

In [33]:
from geoband.API import *

GetCompasData('PJT001_routestationinfo.csv')

In [34]:
route_station = pd.read_csv('PJT001_routestationinfo.csv')
route_station.shape
route_station.head()

(37831, 7)

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no
0,65286,228000018,10-4,1,용인터미널,228001552,47634.0
1,65287,228000018,10-4,2,용인터미널(경유),277102443,NaN
2,65288,228000018,10-4,3,포브스병원,228000443,29439.0
3,65289,228000018,10-4,4,제일교회,228000665,29881.0
4,65290,228000018,10-4,5,라이프아파트,228000664,29457.0


In [35]:
GetCompasData('PJT001_stations_table.csv')

In [36]:
stations = pd.read_csv('PJT001_stations_table.csv')

In [37]:
stations = stations.dropna()

In [38]:
stations.shape

(19558, 12)

In [39]:
stations = stations[['표준정류장ID', '관할관청','WGS84위도', 'WGS84경도']]
stations.columns = ['station_id', 'govrn', 'lat', 'lng']

In [40]:
df_route_station = pd.merge(route_station, stations, on='station_id', how = 'left')

In [41]:
df_route_station.shape
df_route_station.head()

(39152, 10)

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no,govrn,lat,lng
0,65286,228000018,10-4,1,용인터미널,228001552,47634.0,경기도 용인시,37.232783,127.210100
1,65286,228000018,10-4,1,용인터미널,228001552,47634.0,경기도 용인시,37.232783,127.210100
2,65287,228000018,10-4,2,용인터미널(경유),277102443,NaN,NaN,NaN,NaN
3,65288,228000018,10-4,3,포브스병원,228000443,29439.0,경기도 용인시,37.235267,127.210617
4,65289,228000018,10-4,4,제일교회,228000665,29881.0,경기도 용인시,37.234300,127.213333


In [42]:
df_route_station = df_route_station.drop_duplicates()

In [43]:
df_route_station.shape
df_route_station.head()

(37831, 10)

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no,govrn,lat,lng
0,65286,228000018,10-4,1,용인터미널,228001552,47634.0,경기도 용인시,37.232783,127.210100
2,65287,228000018,10-4,2,용인터미널(경유),277102443,NaN,NaN,NaN,NaN
3,65288,228000018,10-4,3,포브스병원,228000443,29439.0,경기도 용인시,37.235267,127.210617
4,65289,228000018,10-4,4,제일교회,228000665,29881.0,경기도 용인시,37.234300,127.213333
5,65290,228000018,10-4,5,라이프아파트,228000664,29457.0,경기도 용인시,37.231483,127.213767


In [44]:
df_route_station = df_route_station[df_route_station['govrn'] == '경기도 화성시']

In [69]:
df_route_station

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no,govrn,lat,lng
998,68834,228000204,24,53,한원IC,233000169,37403.0,경기도 화성시,37.152733,127.122883
999,68835,228000204,24,54,장지동,233000168,36808.0,경기도 화성시,37.155117,127.117900
1000,68836,228000204,24,55,장지동,233000167,36809.0,경기도 화성시,37.155867,127.115017
1003,68839,228000204,24,58,꽃가마약수터,233000166,36810.0,경기도 화성시,37.159100,127.099350
1025,68861,228000204,24,80,꽃가마약수터,233001323,36943.0,경기도 화성시,37.158900,127.098750
...,...,...,...,...,...,...,...,...,...,...
39139,191742,241491011,38-1,55,융건릉,233000600,36262.0,경기도 화성시,37.208050,126.988700
39140,191743,241491011,38-1,56,안녕4통,233002234,55122.0,경기도 화성시,37.212633,126.982783
39141,191744,241491011,38-1,57,수원대학교,233000576,36204.0,경기도 화성시,37.214167,126.979150
39142,191745,241491011,38-1,58,와우2리,233000945,36194.0,경기도 화성시,37.215983,126.976317


In [45]:
# ods[route_id] 에 route_id 노선의 전 구간을 set 에 담습니다.
ods = {}
grp = df_route_station.groupby('bus_line_no')

for route_id, gdf in grp:
    gdf = gdf.sort_values('bus_line_no_seq')
    od_set = set()
    for i in range(1, len(gdf)):
        od = (gdf['station_id'].iloc[i-1], gdf['station_id'].iloc[i])
        od_set.add(od)
    ods[route_id] = od_set

In [46]:
# diff_routes[route_id1][route_id2] 에 노선 route_id1와 route_id2 간의 중복도 값을 구합니다.
diff_routes = {}
for route_id1, od_set1 in ods.items():
    d = {}
    
    for route_id2, od_set2 in ods.items():
        diff = od_set1 & od_set2
        d[route_id2] = len(diff) / len(od_set1)
    diff_routes[route_id1] = d

In [47]:
# 위에서 구한 두 노선의 중복도 값을 데이터프레임으로 만들어줍니다.
# 0~1의 값을 0~100(%) 로 바꿔줍니다.
df_redundancy = pd.DataFrame(diff_routes)
df_redundancy = df_redundancy * 100
df_redundancy.head()

,1,1-1,10,10-1,10-2,10-3,10-4,10-5,10-6,10-7,...,공영3(지화2리),공영3(청원.삼존),공영4(도말.고포),공영4(돌내.형도),공영4(돌내독지1),공영4(돌내독지2),공영4(마산.고포),공영4(삼존2.3리),공영4(솔미.구렛),공영4(형도.고포)
1,100.000000,82.857143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-1,82.857143,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.000000,0.000000,100.000000,57.142857,70.588235,42.105263,52.380952,70.588235,40.740741,45.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10-1,0.000000,0.000000,16.438356,100.000000,94.117647,47.368421,35.714286,55.882353,55.555556,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10-2,0.000000,0.000000,16.438356,76.190476,100.000000,47.368421,35.714286,47.058824,55.555556,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# plt.figure(figsize=(180, 180))
# sns.heatmap(df_redundancy, square=True, annot=True, fmt='.0f', cmap='Blues', cbar=False, vmin=0, vmax=100)
# plt.show()

In [50]:
redundancy_flat = df_redundancy.values.flatten()
redundancy_flat = [i for i in redundancy_flat if i != 100]

In [51]:
pd.Series(redundancy_flat).iplot(kind='hist')

In [52]:
pd.Series(redundancy_flat).describe()

count    42919.000000
mean         3.026240
std         11.397392
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         97.435897
dtype: float64

In [53]:
topn_pair = {}
for route_id1, others in diff_routes.items():
    for route_id2, redundancy in others.items():
        if route_id1 == route_id2:
            continue
        if redundancy >= 0.8 and (topn_pair.get((route_id2, route_id1), None) == None):
            topn_pair[(route_id1, route_id2)] = redundancy

In [54]:
len(topn_pair)

257

In [55]:
# 먼저 위에서 구한 노선쌍의 중복도를 데이터프레임으로 만듭니다.
df_topn_pair = pd.Series(topn_pair).to_frame().reset_index()
df_topn_pair.columns = ['route_1', 'route_2', 'redundancy']
df_topn_pair.sort_values('redundancy', ascending=False, inplace=True)
df_topn_pair

,route_1,route_2,redundancy
128,36-2A,36A,1.0
77,20-2(용포고정천),공영4(돌내.형도),1.0
90,20-2(천등화석지),공영4(돌내독지1),1.0
89,20-2(천등화석지),공영4(돌내.형도),1.0
87,20-2(천등.고정),공영4(돌내독지1),1.0
...,...,...,...
109,25,2-2,0.8
108,25,2-1,0.8
107,25,2,0.8
106,25,18,0.8


In [56]:
temp = df_topn_pair[df_topn_pair['redundancy'] != 1.0]

In [59]:
pd.set_option('display.max_rows', 500)

In [60]:
temp

,route_1,route_2,redundancy
158,공영1(사곳.궁평),공영1(매화.궁평),0.974359
157,공영1(궁평.사곳),공영1(매화.궁평),0.973684
144,6,6-2,0.968750
153,80,81,0.963855
116,31,31-2,0.960000
98,20-3(쌍쏭.대광A),20-3(왕모대시청),0.952381
3,10-1,12-1,0.952381
92,20-3(당성왕모대),공영1(궁평.사곳),0.947368
93,20-3(당성왕모대),공영1(매화.궁평),0.947368
94,20-3(당성왕모대),공영1(사곳.궁평),0.947368


In [98]:
def get_route_path(route_id):
    idx = df_route_station['bus_line_no'] == route_id
    return df_route_station[idx][['lng', 'lat']].values.tolist()

def get_route_name(route_id):
    idx = df_route_station['bus_line_no'] == route_id
    return df_route_station[idx]['bus_line_no']

In [99]:
d = {}
path_list, name_list = [], []

for i, row in temp.iterrows():
    if d.get(row['route_1'], None) == None:
        path_list.append(get_route_path(row['route_1']))
        name_list.append(get_route_name(row['route_1']))
        d[row['route_1']] = True
    if d.get(row['route_2'], None) == None:  
        path_list.append(get_route_path(row['route_2']))
        name_list.append(get_route_name(row['route_2']))
        d[row['route_2']] = True

In [103]:
draw_path(path_list, name_list, colors=[[255, 233, 0]]*len(path_list), opacity=0.01)
# 지도는 안나오고 눈물만 나오네요 